##### Copyright 2019 The TensorFlow Authors.



In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CSV 데이터 불러오기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/csv"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ko/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북 다운로드하기</a>
  </td>
</table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도 불구하고 공식 영문 문서의 내용과 일치하지 않을 수 있습니다. 이 번역에 개선할 부분이 있다면 tensorflow/docs 깃헙 저장소로 풀 리퀘스트를 보내주시기 바랍니다. 문서 번역이나 리뷰에 참여하려면 docs-ko@tensorflow.org로 메일을 보내주시기 바랍니다

이 튜토리얼에서는 CSV 파일을 불러와서 `tf.data.Dataset`로 변환하는 방법을 살펴봅니다.

이 튜토리얼에서 사용된 데이터는 타이타닉 승객 목록입니다. 이 모델은 연령, 성별, 티켓 등급 및 개인 여행 여부와 같은 특성을 기반으로 승객의 생존 가능성을 예측합니다.

## 설치

In [ ]:
try:
  # %tensorflow_version은 코랩에서만 사용할 수 있습니다.
  %tensorflow_version 2.x
except Exception:
  pass


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [ ]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [ ]:
# numpy 값을 읽기 쉽게 표시하도록 만듭니다.
np.set_printoptions(precision=3, suppress=True)

## 데이터 불러오기

먼저 CSV 파일의 상단을 살펴보고, 데이터 형식이 어떻게 지정되었는지 확인해봅시다.

In [ ]:
!head {train_file_path}

[Pandas로 불러오기](pandas.ipynb)한 후에 NumPy 배열을 TensorFlow에 전달해도 됩니다. 큰 파일 모음을 불러와야 하거나, [TensorFlow and tf.data](../../guide/data.ipynb)와 통합된 로더를 사용해야 하는 경우에는 `tf.data.experimental.make_csv_dataset` 함수를 사용합니다.

명시적으로 식별해야하는 유일한 열은 모델이 예측하려는 값을 가진 열입니다.

In [ ]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

이제 파일에서 CSV 데이터를 읽고 데이터세트를 만듭니다.

(전체 문서는 `tf.data.experimental.make_csv_dataset`을 참조하십시오.)

In [ ]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # 예제를 좀더 쉽게 살펴볼 수 있도록 일부러 작게 설정했습니다.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [ ]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

데이터세트의 각 항목은 배치이며 (*많은 예*, *많은 레이블*)와 같은 튜플로 표시됩니다. 예제의 데이터는 행 기반 텐서가 아닌 열 기반 텐서로 구성되며, 각 텐서는 배치 크기(이 경우는 5)만큼의 요소가 있습니다.

직접 보는 것이 이해하는데 더 도움이 될 것입니다.

In [ ]:
show_batch(raw_train_data)

보이는 바와 같이 CSV의 열 이름이 지정됩니다. 데이터세트 생성자가 이름을 자동으로 선택합니다. 작업중인 파일의 첫 번째 행이 열 이름이 아니라면 `make_csv_dataset` 함수의`column_names` 인수에 문자열 리스트로 이름을 전달합니다.

In [ ]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

이 예에서는 사용 가능한 모든 열을 사용합니다. 데이터세트에서 일부 열을 생략해야 하는 경우에는 사용하려는 열의 목록을 작성하고 생성자의 `select_columns`인수(선택적)에 전달하면 됩니다.

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

## 데이터 전처리

CSV 파일은 다양한 데이터 유형을 포함할 수 있습니다. 일반적으로 데이터를 모델에 전달하기 전에 이러한 혼합 유형들을 고정 길이 벡터로 변환하고자 할 것입니다.

TensorFlow에는 일반적인 입력 변환을 위한 내장 시스템이 있습니다 :`tf.feature_column`. 자세한 내용은 [이 튜토리얼] (../ keras / feature_columns)을 참조하십시오.


([nltk](https://www.nltk.org/)이나 [sklearn](https://scikit-learn.org/stable/))과 같은 도구로 데이터를 전처리하고, 처리된 데이터를 TensorFlow에 전달하면 됩니다.


모델 내에서 전처리를 할 때의 주요 이점은 전처리 과정을 모델에 포함시킬 수 있다는 것입니다. 이렇게 하면 원시 데이터를 모델에 직접 전달해도 됩니다.

### 연속형 데이터

데이터가 이미 적절한 숫자 형식이라면 데이터를 모델에 전달하기 전에 벡터로 묶을 수 있습니다:

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

In [ ]:
example_batch, labels_batch = next(iter(temp_dataset)) 

다음은 모든 열을 묶는 간단한 함수입니다:

In [ ]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

이 함수를 데이터세트의 각 요소에 적용합니다:

In [ ]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

혼합 데이터 유형이 있는 경우 이러한 단순 숫자 필드를 분리할 수 있습니다. 이를 `tf.feature_column` api로 처리할 수도 있지만 약간의 오버 헤드를 발생시키므로, 정말 필요한 경우가 아니라면 피하는 것이 좋습니다. 혼합 데이터세트로 되돌아가겠습니다.

In [ ]:
show_batch(raw_train_data)

In [ ]:
example_batch, labels_batch = next(iter(temp_dataset)) 

따라서 숫자 특성 리스트를 선택하고 이를 단일 열로 묶는 좀더 일반적인 전처리기를 정의합니다.

In [ ]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [ ]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [ ]:
show_batch(packed_train_data)

In [ ]:
example_batch, labels_batch = next(iter(packed_train_data)) 

#### 데이터 정규화

연속형 데이터는 항상 정규화되어야 합니다.

In [ ]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

In [ ]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [ ]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

이제 숫자 열을 만듭니다. `tf.feature_columns.numeric_column`API는 `normalizer_fn`인수를 받는데, 이 함수는 각 배치마다 실행될 것입니다.

[`functools.partial`](https://docs.python.org/3/library/functools.html#functools.partial)을 사용하여 `MEAN`과 `STD`를 normalizer_fn에 바인딩합니다.

In [ ]:
# 생성한 것을 바로 확인해 보세요.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

모델을 학습시킬 때 이 피쳐 열을 포함시키고, 이 숫자 데이터 블록을 가운데에 배치합니다:

In [ ]:
example_batch['numeric']

In [ ]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

여기서 사용된 평균 기반 정규화는 각 열의 평균을 미리 알아야합니다.

### 범주형 데이터

CSV 데이터의 일부 열은 범주형 열입니다. 즉, 컨텐츠는 제한된 옵션 중의 하나여야 합니다.

`tf.feature_column`API를 사용하여 각 범주형 열에 대해 `tf.feature_column.indicator_column`형의 컬렉션을 생성합니다.

In [ ]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [ ]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [ ]:
# 방금 생성한 것을 확인해 봅니다.
categorical_columns

In [ ]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

이것은 나중에 모델을 만들 때 데이터 처리 입력의 일부가 됩니다.

### 결합된 전처리층

두 개의 피처 열 컬렉션을 결합하고 이를 `tf.keras.layers.DenseFeatures`에 전달하여 입력층을 만듭니다. 이 입력층으로 두 개의 입력 유형을 추출하고 전처리할 것입니다.

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [ ]:
print(preprocessing_layer(example_batch).numpy()[0])

## 모델 만들기

`preprocessing_layer`부터 시작하는 `tf.keras.Sequential`을 만듭니다.

In [ ]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

## 훈련, 평가 및 예측

이제 모델을 인스턴스화하고 훈련시킬 수 있습니다.

In [ ]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [ ]:
model.fit(train_data, epochs=20)

모델이 학습되면 `test_data` 세트에서 정확도를 확인할 수 있습니다.

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

`tf.keras.Model.predict`를 사용하여 배치 또는 배치 데이터세트로부터 레이블을 추론합니다.

In [ ]:
predictions = model.predict(test_data)

# 몇 개 결과 표시
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

